# GraphCast

This colab lets you run several versions of GraphCast.

The model weights, normalization statistics, and example inputs are available on [Google Cloud Bucket](https://console.cloud.google.com/storage/browser/dm_graphcast).

A Colab runtime with TPU/GPU acceleration will substantially speed up generating predictions and computing the loss/gradients. If you're using a CPU-only runtime, you can switch using the menu "Runtime > Change runtime type".

> <p><small><small>Copyright 2023 DeepMind Technologies Limited.</small></p>
> <p><small><small>Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at <a href="http://www.apache.org/licenses/LICENSE-2.0">http://www.apache.org/licenses/LICENSE-2.0</a>.</small></small></p>
> <p><small><small>Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.</small></small></p>

# Installation and Initialization


In [4]:
# @title Pip install graphcast and dependencies

# %pip install --upgrade https://github.com/deepmind/graphcast/archive/master.zip

In [5]:
# @title Workaround for cartopy crashes

# Workaround for cartopy crashes due to the shapely installed by default in
# google colab kernel (https://github.com/anitagraser/movingpandas/issues/81):
# !pip uninstall -y shapely
# !pip install shapely --no-binary shapely

In [1]:
# @title Imports

import dataclasses
import datetime
import functools
import math
import re
from typing import Optional

import cartopy.crs as ccrs
from google.cloud import storage
from graphcast import autoregressive
from graphcast import casting
from graphcast import checkpoint
from graphcast import data_utils
from graphcast import graphcast
from graphcast import normalization
from graphcast import rollout
from graphcast import xarray_jax
from graphcast import xarray_tree
from IPython.display import HTML
import ipywidgets as widgets
import haiku as hk
import jax
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
import xarray


def parse_file_parts(file_name):
  return dict(part.split("-", 1) for part in file_name.split("_"))


In [2]:
# @title Authenticate with Google Cloud Storage

gcs_client = storage.Client.create_anonymous_client()
gcs_bucket = gcs_client.get_bucket("dm_graphcast")
gcs_bucket

<Bucket: dm_graphcast>

In [3]:
# @title Plotting functions

# function: data에서 variable을 선택해서 pressure level, time max_step 선택해서 작은 data 추출
def select(
    data: xarray.Dataset,
    variable: str,
    level: Optional[int] = None,
    max_steps: Optional[int] = None
    ) -> xarray.Dataset:
    
    # 선택한 변수만 남기기
    data = data[variable]
    
    # "batch" 차원이 있으면 첫 번째 배치 선택
    if "batch" in data.dims:
        data = data.isel(batch=0)
        
    # max_steps가 설정되어 있고, "time" 차원이 있으면 시간 스텝 제한
    if max_steps is not None and "time" in data.sizes and max_steps < data.sizes["time"]:
        data = data.isel(time=range(0, max_steps))
        
    # level이 설정되어 있고, "level" 좌표가 있으면 해당 레벨만 선택
    if level is not None and "level" in data.coords:
        data = data.sel(level=level)
        
    return data

# function: data에서 subplot한 걸 figure 1개에 plot
def plot_data(
    data: dict[str, xarray.Dataset],
    fig_title: str,
    plot_size: float = 5, # subplot 크기
    robust: bool = False, # colorbar 자동 조정
    cols: int = 4 # Subplot col 수
    ) -> tuple[xarray.Dataset, matplotlib.colors.Normalize, str]:

    # 첫 번째 데이터셋에서 시간 스텝 수 확인
    first_data = next(iter(data.values()))[0]
    max_steps = first_data.sizes.get("time", 1)
    assert all(max_steps == d.sizes.get("time", 1) for d, _, _ in data.values())

    # subplot의 열과 행 수 계산
    cols = min(cols, len(data))
    rows = math.ceil(len(data) / cols)

    # figure 설정
    figure = plt.figure(figsize=(plot_size * 2 * cols, plot_size * rows))
    figure.suptitle(fig_title, fontsize=16)
    figure.subplots_adjust(wspace=0, hspace=0)
    figure.tight_layout()

    images = []
    for i, (title, (plot_data, norm, cmap)) in enumerate(data.items()):
        # 각 subplot 생성
        ax = figure.add_subplot(rows, cols, i + 1)
        ax.set_xticks([])  # x축 눈금 제거
        ax.set_yticks([])  # y축 눈금 제거
        ax.set_title(title)  # subplot 제목 설정

        # 데이터 시각화
        im = ax.imshow(
            plot_data.isel(time=0, missing_dims="ignore"), norm=norm,
            origin="lower", cmap=cmap)
        
        # colorbar 추가
        plt.colorbar(
            mappable=im,
            ax=ax,
            orientation="vertical",
            pad=0.02,
            aspect=16,
            shrink=0.75,
            cmap=cmap,
            extend=("both" if robust else "neither"))
        images.append(im)

    def update(frame):
        # 애니메이션의 각 프레임 업데이트
        if "time" in first_data.dims:
            td = datetime.timedelta(microseconds=first_data["time"][frame].item() / 1000)
            figure.suptitle(f"{fig_title}, {td}", fontsize=16)
        else:
            figure.suptitle(fig_title, fontsize=16)
        for im, (plot_data, norm, cmap) in zip(images, data.values()):
            im.set_data(plot_data.isel(time=frame, missing_dims="ignore"))

    # 애니메이션 생성
    ani = animation.FuncAnimation(
        fig=figure, func=update, frames=max_steps, interval=250)
    
    # figure 닫기
    plt.close(figure.number)
    
    # 애니메이션을 HTML로 변환하여 반환
    return HTML(ani.to_jshtml())

# function: 데이터를 스케일 조정하고, 컬러 맵을 설정하여 반환
def scale(
    data: xarray.Dataset,
    center: Optional[float] = None,
    robust: bool = False,
    ) -> tuple[xarray.Dataset, matplotlib.colors.Normalize, str]:
    
    # 데이터의 최소값과 최대값 계산
    vmin = np.nanpercentile(data, (2 if robust else 0))
    vmax = np.nanpercentile(data, (98 if robust else 100))
    
    # 중심값이 설정되어 있으면 범위 조정
    if center is not None:
        diff = max(vmax - center, center - vmin)
        vmin = center - diff
        vmax = center + diff
    
    # 데이터와 정규화 객체, 컬러 맵 반환
    return (data, matplotlib.colors.Normalize(vmin, vmax),
            ("RdBu_r" if center is not None else "viridis"))

# Load the Data and initialize the model

## Load the model params

Choose one of the two ways of getting model params:
- **random**: You'll get random predictions, but you can change the model architecture, which may run faster or fit on your device. -> 학습시키고 싶으면 이걸로 하면 됨
- **checkpoint**: You'll get sensible predictions, but are limited to the model architecture that it was trained with, which may not fit on your device. In particular generating gradients uses a lot of memory, so **you'll need at least 25GB of ram** (TPUv4 or A100). -> param. 가지고 시작

Checkpoints vary across a few axes:
- **The mesh size** specifies the internal graph representation of the earth. Smaller meshes will run faster but will have worse outputs. The mesh size does not affect the number of parameters of the model.
- **The resolution and number of pressure levels** must match the data. Lower resolution and fewer levels will run a bit faster. *Data resolution only affects the encoder/decoder(?)*.
- All our models predict precipitation. However, ERA5 includes precipitation, while HRES does not. Our models marked as "ERA5" take precipitation as input and expect ERA5 data as input, while model marked "ERA5-HRES" do not take precipitation as input and are specifically trained to take HRES-fc0 as input (see the data section below).

We provide three pre-trained models.
1. `GraphCast`, the high-resolution model used in the GraphCast paper (0.25 degree resolution, 37 pressure levels), trained on ERA5 data from 1979 to 2017,

2. `GraphCast_small`, a smaller, low-resolution version of GraphCast (1 degree resolution, 13 pressure levels, and a smaller mesh), trained on ERA5 data from 1979 to 2015, useful to run a model **with lower memory and compute constraints**,

3. `GraphCast_operational`, a high-resolution model (0.25 degree resolution, 13 pressure levels) **pre-trained** on ERA5 data from 1979 to 2017 and fine-tuned on HRES data from 2016 to 2021. This model can be initialized from HRES data (does not require precipitation inputs).


In [9]:
# @title Choose the model

# 옵션 리스트 생성: Google Cloud Storage 버킷에서 "params/" 접두어를 가진 블롭(파일) 이름을 리스트에 저장.
# :=는 3.8부터 사용 가능
params_file_options = [
    name for blob in gcs_bucket.list_blobs(prefix="params/")  # 'params/' 접두어를 가진 모든 블롭을 반복
    if (name := blob.name.removeprefix("params/"))]  # 블롭 이름에서 'params/'를 제거하고, 'name' 변수에 할당

# 무작위 초기화 옵션을 위한 위젯 설정:
# 메시 크기를 선택하는 슬라이더 (최소 4, 최대 6)
random_mesh_size = widgets.IntSlider(
    value=4, min=4, max=6, description="Mesh size:")

# GNN 메시지 단계를 선택하는 슬라이더 (최소 1, 최대 32)
random_gnn_msg_steps = widgets.IntSlider(
    value=4, min=1, max=32, description="GNN message steps:")

# 잠재 크기를 선택하는 드롭다운 (2^4 부터 2^9 까지의 값 선택 가능)
random_latent_size = widgets.Dropdown(
    options=[int(2**i) for i in range(4, 10)], value=32, description="Latent size:")

# 압력 레벨을 선택하는 드롭다운 (13 또는 37 선택 가능)
random_levels = widgets.Dropdown(
    options=[13, 37], value=13, description="Pressure levels:")

# 체크포인트 파일을 선택하는 드롭다운 위젯
params_file = widgets.Dropdown(
    options=params_file_options,
    description="Params file:",
    layout={"width": "max-content"})

# 탭 위젯 생성: "Random" 및 "Checkpoint" 탭을 포함
source_tab = widgets.Tab([
    widgets.VBox([
        random_mesh_size,
        random_gnn_msg_steps,
        random_latent_size,
        random_levels,
    ]),
    params_file,
])

# 각 탭에 제목 설정
source_tab.set_title(0, "Random")
source_tab.set_title(1, "Checkpoint")

# 최종 레이아웃 설정: 탭 위젯 및 레이블 포함
widgets.VBox([
    source_tab,
    widgets.Label(value="Run the next cell to load the model. Rerunning this cell clears your selection.")
])

In [11]:
# @title Load the model

# 사용자가 선택한 소스를 가져옵니다. ("Random" 또는 "Checkpoint")
source = source_tab.get_title(source_tab.selected_index)

# 사용자가 "Random" 옵션을 선택한 경우:
if source == "Random":
    params = None  # 모델 파라미터는 나중에 설정됩니다.
    state = {}  # 초기 상태는 빈 딕셔너리로 설정됩니다.
    
    # GraphCast 모델 구성 설정
    model_config = graphcast.ModelConfig(
        resolution=0,  # 해상도는 0으로 설정
        mesh_size=random_mesh_size.value,  # 사용자가 선택한 메시 크기
        latent_size=random_latent_size.value,  # 사용자가 선택한 잠재 크기
        gnn_msg_steps=random_gnn_msg_steps.value,  # 사용자가 선택한 GNN 메시지 단계
        hidden_layers=1,  # 은닉층 수는 1로 설정
        radius_query_fraction_edge_length=0.6  # 반경 쿼리 비율
    )
    
    # GraphCast 작업 구성 설정
    task_config = graphcast.TaskConfig(
        input_variables=graphcast.TASK.input_variables,  # 입력 변수
        target_variables=graphcast.TASK.target_variables,  # 목표 변수
        forcing_variables=graphcast.TASK.forcing_variables,  # 강제 변수
        pressure_levels=graphcast.PRESSURE_LEVELS[random_levels.value],  # 사용자가 선택한 압력 레벨
        input_duration=graphcast.TASK.input_duration,  # 입력 지속 시간
    )

# 사용자가 "Checkpoint" 옵션을 선택한 경우:
else:
    assert source == "Checkpoint"  # 소스가 "Checkpoint"인지 확인
    
    # 선택한 체크포인트 파일을 Google Cloud Storage에서 열기
    with gcs_bucket.blob(f"params/{params_file.value}").open("rb") as f:
        ckpt = checkpoint.load(f, graphcast.CheckPoint)  # 체크포인트 로드
    
    params = ckpt.params  # 로드된 체크포인트에서 파라미터 가져오기
    state = {}  # 초기 상태는 빈 딕셔너리로 설정
    
    # 체크포인트에서 모델 구성 가져오기
    model_config = ckpt.model_config
    # 체크포인트에서 작업 구성 가져오기
    task_config = ckpt.task_config
    
    # 모델 설명 출력
    print("Model description:\n", ckpt.description, "\n")
    # 모델 라이선스 출력
    print("Model license:\n", ckpt.license, "\n")

# 최종 모델 구성 반환
model_config

Model description:
 
GraphCast model at 0.25deg resolution, with 37 pressure levels. This model is
trained on ERA5 data from 1979 to 2017, and can be causally evaluated on 2018
and later years. This model takes as inputs `total_precipitation_6hr`. This was
described in the paper
`GraphCast: Learning skillful medium-range global weather forecasting`
(https://arxiv.org/abs/2212.12794).
 

Model license:
 
The model weights are licensed under the Creative Commons
Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0). You
may obtain a copy of the License at:
https://creativecommons.org/licenses/by-nc-sa/4.0/.
The weights were trained on ERA5 data, see README for attribution statement.
 



ModelConfig(resolution=0.25, mesh_size=6, latent_size=512, gnn_msg_steps=16, hidden_layers=1, radius_query_fraction_edge_length=0.5999912857713345, mesh2grid_edge_normalization_factor=0.6180338738074472)

## Load the example data

Several example datasets are available, varying across a few axes:
- **Source**: fake, era5, hres
- **Resolution**: 0.25deg, 1deg, 6deg
- **Levels**: 13, 37
- **Steps**: How many timesteps are included

Not all combinations are available.
- Higher resolution is only available for fewer steps due to the memory requirements of loading them.
- HRES is only available in 0.25 deg, with 13 pressure levels.

The data resolution must match the model that is loaded.

Some transformations were done from the base datasets:
- **We accumulated precipitation over 6 hours instead of the default 1 hour.**
- For HRES data, each time step corresponds to the HRES forecast at leadtime 0, essentially providing an "initialisation" from HRES. See HRES-fc0 in the GraphCast paper for further description. Note that a 6h accumulation of precipitation is not available from HRES, so our model taking HRES inputs does not depend on precipitation. However, because our models predict precipitation, we include the ERA5 precipitation in the example data so it can serve as an illustrative example of ground truth.
- We include ERA5 `toa_incident_solar_radiation` in the data. Our model uses the radiation at -6h, 0h and +6h as a forcing term for each 1-step prediction. If the radiation is missing from the data (e.g. in an operational setting), it will be computed using a custom implementation that produces values similar to those in ERA5.

In [12]:
# @title Get and filter the list of available example datasets

# Google Cloud Storage 버킷에서 "dataset/" 접두어를 가진 블롭(파일) 이름을 리스트로 저장
dataset_file_options = [
    name for blob in gcs_bucket.list_blobs(prefix="dataset/")
    if (name := blob.name.removeprefix("dataset/"))]  # "dataset/" 접두어를 제거하여 이름을 저장

# 모델과 데이터셋의 유효성을 검사하는 함수
def data_valid_for_model(
    file_name: str, 
    model_config: graphcast.ModelConfig, 
    task_config: graphcast.TaskConfig):
    # 파일 이름에서 접미사 ".nc"를 제거하고 파일의 각 부분을 파싱
    file_parts = parse_file_parts(file_name.removesuffix(".nc"))
    
    # 모델의 설정과 데이터셋 파일의 설정을 비교하여 유효성 검사
    return (
        # 모델의 해상도가 0이거나 파일의 해상도와 일치해야 함
        model_config.resolution in (0, float(file_parts["res"])) and
        # 모델의 압력 레벨 수가 파일의 압력 레벨 수와 일치해야 함
        len(task_config.pressure_levels) == int(file_parts["levels"]) and
        (
            # 모델이 강수량을 입력 변수로 가지고 있는 경우, 파일 소스는 "era5" 또는 "fake"여야 함
            ("total_precipitation_6hr" in task_config.input_variables and
             file_parts["source"] in ("era5", "fake")) or
            # 모델이 강수량을 입력 변수로 가지고 있지 않은 경우, 파일 소스는 "hres" 또는 "fake"여야 함
            ("total_precipitation_6hr" not in task_config.input_variables and
             file_parts["source"] in ("hres", "fake"))
        )
    )

# 데이터셋 파일을 선택할 수 있는 드롭다운 위젯
dataset_file = widgets.Dropdown(
    options=[
        # 파일 이름에서 파싱한 각 부분을 문자열로 결합하여 옵션으로 제공
        (", ".join([f"{k}: {v}" for k, v in parse_file_parts(option.removesuffix(".nc")).items()]), option)
        # 모델 설정에 유효한 데이터셋 파일만 필터링
        for option in dataset_file_options
        if data_valid_for_model(option, model_config, task_config)
    ],
    description="Dataset file:",
    layout={"width": "max-content"}
)

# 선택한 데이터셋 파일을 표시하는 레이아웃
widgets.VBox([
    dataset_file,
    widgets.Label(value="Run the next cell to load the dataset. Rerunning this cell clears your selection and refilters the datasets that match your model.")
])


In [13]:
# @title Load weather data

# 데이터셋 파일이 모델 설정에 유효한지 확인
if not data_valid_for_model(dataset_file.value, model_config, task_config):
    raise ValueError(
        "Invalid dataset file, rerun the cell above and choose a valid dataset file.")

# 유효한 경우 Google Cloud Storage에서 데이터셋 파일을 열고 로드
# with gcs_bucket.blob(f"dataset/{dataset_file.value}").open("rb") as f:
#     example_batch = xarray.load_dataset(f).compute()
example_batch=xarray.open_dataset('testdata/source-era5_date-2022-01-01_res-0.25_levels-37_steps-12.nc')

# 데이터셋의 시간 차원이 최소 3인지 확인 (입력용 2, 목표용 1 이상)
assert example_batch.dims["time"] >= 3  # 2 for input, >=1 for targets

# 선택한 데이터셋 파일의 정보를 출력
print(", ".join([f"{k}: {v}" for k, v in parse_file_parts(dataset_file.value.removesuffix(".nc")).items()]))

# 로드된 데이터셋을 출력
example_batch


source: era5, date: 2022-01-01, res: 0.25, levels: 37, steps: 01


/tmp/ipykernel_762227/723201860.py:14: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  assert example_batch.dims["time"] >= 3  # 2 for input, >=1 for targets


<xarray.Dataset> Size: 13GB
Dimensions:                       (lon: 1440, lat: 721, level: 37, time: 14,
                                   batch: 1)
Coordinates:
  * lon                           (lon) float32 6kB 0.0 0.25 0.5 ... 359.5 359.8
  * lat                           (lat) float32 3kB -90.0 -89.75 ... 89.75 90.0
  * level                         (level) int32 148B 1 2 3 5 ... 950 975 1000
  * time                          (time) timedelta64[ns] 112B 0 days 00:00:00...
    datetime                      (batch, time) datetime64[ns] 112B ...
Dimensions without coordinates: batch
Data variables: (12/14)
    geopotential_at_surface       (lat, lon) float32 4MB ...
    land_sea_mask                 (lat, lon) float32 4MB ...
    2m_temperature                (batch, time, lat, lon) float32 58MB ...
    mean_sea_level_pressure       (batch, time, lat, lon) float32 58MB ...
    10m_v_component_of_wind       (batch, time, lat, lon) float32 58MB ...
    10m_u_component_of_wind       (batch, time, lat, lon) float32 58MB ...
    ...                            ...
    temperature                   (batch, time, level, lat, lon) float32 2GB ...
    geopotential                  (batch, time, level, lat, lon) float32 2GB ...
    u_component_of_wind           (batch, time, level, lat, lon) float32 2GB ...
    v_component_of_wind           (batch, time, level, lat, lon) float32 2GB ...
    vertical_velocity             (batch, time, level, lat, lon) float32 2GB ...
    specific_humidity             (batch, time, level, lat, lon) float32 2GB ...

In [14]:
# @title Choose data to plot

# 플로팅할 변수를 선택하는 드롭다운 위젯
plot_example_variable = widgets.Dropdown(
    options=example_batch.data_vars.keys(),  # 데이터셋의 변수 키를 옵션으로 제공
    value="2m_temperature",  # 기본 선택 변수
    description="Variable"  # 설명 레이블
)

# 플로팅할 압력 레벨을 선택하는 드롭다운 위젯
plot_example_level = widgets.Dropdown(
    options=example_batch.coords["level"].values,  # 데이터셋의 압력 레벨 값을 옵션으로 제공
    value=500,  # 기본 선택 압력 레벨
    description="Level"  # 설명 레이블
)

# 강건한 플로팅 옵션을 설정하는 체크박스 위젯
plot_example_robust = widgets.Checkbox(
    value=True,  # 기본적으로 선택된 상태
    description="Robust"  # 설명 레이블
)

# 최대 시간 단계를 설정하는 슬라이더 위젯
plot_example_max_steps = widgets.IntSlider(
    min=1,  # 최소값
    max=example_batch.dims["time"],  # 데이터셋의 시간 차원의 크기를 최대값으로 설정
    value=example_batch.dims["time"],  # 기본적으로 전체 시간 단계를 선택
    description="Max steps"  # 설명 레이블
)

# 인터페이스 레이아웃을 설정하는 VBox 위젯
widgets.VBox([
    plot_example_variable,  # 변수 선택 드롭다운
    plot_example_level,  # 레벨 선택 드롭다운
    plot_example_robust,  # 강건한 플로팅 체크박스
    plot_example_max_steps,  # 최대 시간 단계 슬라이더
    widgets.Label(value="Run the next cell to plot the data. Rerunning this cell clears your selection.")  # 설명 레이블
])


/tmp/ipykernel_762227/4214671020.py:26: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  max=example_batch.dims["time"],  # 데이터셋의 시간 차원의 크기를 최대값으로 설정
/tmp/ipykernel_762227/4214671020.py:27: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  value=example_batch.dims["time"],  # 기본적으로 전체 시간 단계를 선택


In [15]:
# @title Choose training and eval data to extract

# Create a slider widget for selecting the number of training steps
# The slider ranges from 1 to (example_batch.sizes["time"] - 2) with an initial value of 1
train_steps = widgets.IntSlider(
    value=1, 
    min=1, 
    max=example_batch.sizes["time"] - 2, 
    description="Train steps"
)

# Create a slider widget for selecting the number of evaluation steps
# The slider ranges from 1 to (example_batch.sizes["time"] - 2) with an initial value equal to the max value
eval_steps = widgets.IntSlider(
    value=example_batch.sizes["time"] - 2, 
    min=1, 
    max=example_batch.sizes["time"] - 2, 
    description="Eval steps"
)

# Arrange the widgets in a vertical box layout
# The VBox contains the train_steps slider, eval_steps slider, and a label with instructions
widgets.VBox([
    train_steps,
    eval_steps,
    widgets.Label(value="Run the next cell to extract the data. Rerunning this cell clears your selection.")
])

In [20]:
# @title Extract training and eval data

# Extract training data: inputs, targets, and forcings
# The extraction is based on the specified target lead times from 6 hours to train_steps.value * 6 hours
train_inputs, train_targets, train_forcings = data_utils.extract_inputs_targets_forcings(
    example_batch, 
    target_lead_times=slice("6h", f"{train_steps.value * 6}h"), 
    **dataclasses.asdict(task_config)
)

# Extract evaluation data: inputs, targets, and forcings
# The extraction is based on the specified target lead times from 6 hours to eval_steps.value * 6 hours
eval_inputs, eval_targets, eval_forcings = data_utils.extract_inputs_targets_forcings(
    example_batch, 
    target_lead_times=slice("6h", f"{eval_steps.value * 6}h"), 
    **dataclasses.asdict(task_config)
)

# Print the dimensions of the example batch and extracted training and evaluation data
print("All Examples:  ", example_batch.dims.mapping)
print("Train Inputs:  ", train_inputs.dims.mapping)
print("Train Targets: ", train_targets.dims.mapping)
print("Train Forcings:", train_forcings.dims.mapping)
print("Eval Inputs:   ", eval_inputs.dims.mapping)
print("Eval Targets:  ", eval_targets.dims.mapping)
print("Eval Forcings: ", eval_forcings.dims.mapping)
eval_forcings.isel(time=slice(0,2))

|| data_utils.py -> add_derived_vars() init. ||
|| data_utils.py -> add_derived_vars() init. ||
All Examples:   {'lon': 1440, 'lat': 721, 'level': 37, 'time': 14, 'batch': 1}
Train Inputs:   {'batch': 1, 'time': 2, 'lat': 721, 'lon': 1440, 'level': 37}
Train Targets:  {'batch': 1, 'time': 1, 'lat': 721, 'lon': 1440, 'level': 37}
Train Forcings: {'batch': 1, 'time': 1, 'lat': 721, 'lon': 1440}
Eval Inputs:    {'batch': 1, 'time': 2, 'lat': 721, 'lon': 1440, 'level': 37}
Eval Targets:   {'batch': 1, 'time': 12, 'lat': 721, 'lon': 1440, 'level': 37}
Eval Forcings:  {'batch': 1, 'time': 12, 'lat': 721, 'lon': 1440}


<xarray.Dataset> Size: 8MB
Dimensions:                       (batch: 1, time: 2, lat: 721, lon: 1440)
Coordinates:
  * lon                           (lon) float32 6kB 0.0 0.25 0.5 ... 359.5 359.8
  * lat                           (lat) float32 3kB -90.0 -89.75 ... 89.75 90.0
  * time                          (time) timedelta64[ns] 16B 06:00:00 12:00:00
Dimensions without coordinates: batch
Data variables:
    toa_incident_solar_radiation  (batch, time, lat, lon) float32 8MB ...
    year_progress_sin             (batch, time) float32 8B 0.01559 0.01989
    year_progress_cos             (batch, time) float32 8B 0.9999 0.9998
    day_progress_sin              (batch, time, lon) float32 12kB -8.742e-08 ...
    day_progress_cos              (batch, time, lon) float32 12kB -1.0 ... -0...

In [12]:
# @title Load normalization data

# Load the dataset containing the standard deviations of differences by level
# from the specified Google Cloud Storage (GCS) bucket
with gcs_bucket.blob("stats/diffs_stddev_by_level.nc").open("rb") as f:
    diffs_stddev_by_level = xarray.load_dataset(f).compute()

# Load the dataset containing the mean values by level
# from the specified GCS bucket
with gcs_bucket.blob("stats/mean_by_level.nc").open("rb") as f:
    mean_by_level = xarray.load_dataset(f).compute()

# Load the dataset containing the standard deviations by level
# from the specified GCS bucket
with gcs_bucket.blob("stats/stddev_by_level.nc").open("rb") as f:
    stddev_by_level = xarray.load_dataset(f).compute()

In [13]:
# @title Build jitted functions, and possibly initialize random weights

# This function initializes and wraps the GraphCast predictor, applying BFloat16 casting, normalization, and autoregressive wrapping.
def construct_wrapped_graphcast(
    model_config: graphcast.ModelConfig,
    task_config: graphcast.TaskConfig):
  """Constructs and wraps the GraphCast Predictor."""
  # Deeper one-step predictor.
  # Initialize the GraphCast predictor with the provided model and task configurations.
  predictor = graphcast.GraphCast(model_config, task_config)

  # Modify inputs/outputs to `graphcast.GraphCast` to handle conversion to
  # from/to float32 to/from BFloat16.
  # Wrap the predictor to handle conversion between float32 and BFloat16.
  predictor = casting.Bfloat16Cast(predictor)

  # Modify inputs/outputs to `casting.Bfloat16Cast` so the casting to/from
  # BFloat16 happens after applying normalization to the inputs/targets.
  # Wrap the predictor to apply normalization to inputs/targets before casting to/from BFloat16.
  predictor = normalization.InputsAndResiduals(
      predictor,
      diffs_stddev_by_level=diffs_stddev_by_level,
      mean_by_level=mean_by_level,
      stddev_by_level=stddev_by_level)

  # Wraps everything so the one-step model can produce trajectories.
  predictor = autoregressive.Predictor(predictor, gradient_checkpointing=True)
  return predictor

# This function defines the forward pass of the model using the constructed predictor.
@hk.transform_with_state
def run_forward(model_config, task_config, inputs, targets_template, forcings):
    """Runs the forward pass using the constructed predictor."""
    predictor = construct_wrapped_graphcast(model_config, task_config)
    return predictor(inputs, targets_template=targets_template, forcings=forcings)

# This function calculates the loss and diagnostics for the given inputs, targets, and forcings.
@hk.transform_with_state
def loss_fn(model_config, task_config, inputs, targets, forcings):
    """Calculates the loss and diagnostics using the constructed predictor."""
    predictor = construct_wrapped_graphcast(model_config, task_config)
    loss, diagnostics = predictor.loss(inputs, targets, forcings)
    return xarray_tree.map_structure(
        lambda x: xarray_jax.unwrap_data(x.mean(), require_jax=True),
        (loss, diagnostics)
    )

# This function calculates the gradients of the loss function with respect to the parameters.
def grads_fn(params, state, model_config, task_config, inputs, targets, forcings):
  def _aux(params, state, i, t, f):
    (loss, diagnostics), next_state = loss_fn.apply(
        params, state, jax.random.PRNGKey(0), model_config, task_config,
        i, t, f)
    return loss, (diagnostics, next_state)
  (loss, (diagnostics, next_state)), grads = jax.value_and_grad(
      _aux, has_aux=True)(params, state, inputs, targets, forcings)
  return loss, diagnostics, next_state, grads

# Jax doesn't seem to like passing configs as args through the jit. Passing it
# in via partial (instead of capture by closure) forces jax to invalidate the
# jit cache if you change configs.
# Utility function to apply model and task configurations through functools.partial
def with_configs(fn):
  return functools.partial(
      fn, model_config=model_config, task_config=task_config)

# Always pass params and state, so the usage below are simpler
# Ensures functions always receive parameters and state.
def with_params(fn):
  return functools.partial(fn, params=params, state=state)

# Our models aren't stateful, so the state is always empty, so just return the
# predictions. This is requiredy by our rollout code, and generally simpler.
def drop_state(fn):
  return lambda **kw: fn(**kw)[0]

# JIT compile the initialization of the forward pass
init_jitted = jax.jit(with_configs(run_forward.init))

if params is None:
    # Initialize parameters and state if they are not already set
    params, state = init_jitted(
        rng=jax.random.PRNGKey(0),
        inputs=train_inputs,
        targets_template=train_targets,
        forcings=train_forcings
    )

# JIT compile the loss function, gradients function, and forward pass with necessary configurations and parameters
loss_fn_jitted = drop_state(with_params(jax.jit(with_configs(loss_fn.apply))))
grads_fn_jitted = with_params(jax.jit(with_configs(grads_fn)))
run_forward_jitted = drop_state(with_params(jax.jit(with_configs(run_forward.apply))))

# Run the model

Note that the cell below may take a while (possibly minutes) to run the first time you execute them, because this will include the time it takes for the code to compile. The second time running will be significantly faster.

This use the python loop to iterate over prediction steps, where the 1-step prediction is jitted. This has lower memory requirements than the training steps below, and should enable making prediction with the small GraphCast model on 1 deg resolution data for 4 steps.

In [14]:
# @title Autoregressive rollout (loop in python)

# Ensure that the model resolution matches the data resolution
assert model_config.resolution in (0, 360. / eval_inputs.sizes["lon"]), (
    "Model resolution doesn't match the data resolution. You likely want to "
    "re-filter the dataset list, and download the correct data.")

# Print the dimensions of the evaluation inputs, targets, and forcings
print("Inputs:  ", eval_inputs.dims.mapping)
print("Targets: ", eval_targets.dims.mapping)
print("Forcings:", eval_forcings.dims.mapping)

# Perform autoregressive rollout to generate predictions
predictions = rollout.chunked_prediction(
    run_forward_jitted,
    rng=jax.random.PRNGKey(0),
    inputs=eval_inputs,
    targets_template=eval_targets * np.nan,
    forcings=eval_forcings
)

# Display the predictions
predictions

Inputs:   {'batch': 1, 'time': 2, 'lat': 721, 'lon': 1440, 'level': 37}
Targets:  {'batch': 1, 'time': 12, 'lat': 721, 'lon': 1440, 'level': 37}
Forcings: {'batch': 1, 'time': 12, 'lat': 721, 'lon': 1440}
|| rollout.py -> chunked_prediction() init. ||
|| rollout.py -> chunked_prediction_generator() init. ||


/home/hiskim1/graphcast/graphcast/rollout.py:130: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  num_target_steps = targets_template.dims["time"]
/home/hiskim1/graphcast/graphcast/autoregressive.py:202: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  scan_length = targets_template.dims['time']


|| graphcast.py -> __call__() init ||
latent_mesh_nodes shape: (40962, 1, 512)
updated_latent_mesh_nodes shape: (40962, 1, 512)
output_grid_nodes shape: (1038240, 1, 227)
|| graphcast.py -> __call__() done ||


/home/hiskim1/graphcast/graphcast/autoregressive.py:115: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  num_inputs = inputs.dims['time']
2024-07-21 16:07:18.762104: E external/xla/xla/service/slow_operation_alarm.cc:65] Constant folding an instruction is taking > 1s:

  %pad.1 = bf16[40962,1,474]{2,1,0} pad(bf16[40962,1,3]{2,1,0} %constant.348, bf16[] %constant.349), padding=0_0x0_0x471_0, metadata={op_name="jit(<unnamed wrapped function>)/jit(main)/while/body/concatenate[dimension=2]" source_file="/home/hiskim1/graphcast/graphcast/graphcast.py" source_line=643}

This isn't necessarily a bug; constant-folding is inherently a trade-off between compilation time and speed at runtime. XLA has some guards that attempt to keep constant folding from taking too long, but fundamentally you'll alwa

XlaRuntimeError: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 3206250496 bytes.

: 

In [ ]:
# @title Choose predictions to plot

# Dropdown widget to select the variable to plot
# Options are the data variables available in the predictions
plot_pred_variable = widgets.Dropdown(
    options=predictions.data_vars.keys(),
    value="2m_temperature",
    description="Variable"
)

# Dropdown widget to select the level to plot
# Options are the levels available in the predictions
plot_pred_level = widgets.Dropdown(
    options=predictions.coords["level"].values,
    value=500,
    description="Level"
)

# Checkbox widget to enable/disable robust plotting
plot_pred_robust = widgets.Checkbox(
    value=True,
    description="Robust"
)

# Slider widget to select the maximum number of steps to plot
# The range is from 1 to the maximum number of time steps in the predictions
plot_pred_max_steps = widgets.IntSlider(
    min=1,
    max=predictions.dims["time"],
    value=predictions.dims["time"],
    description="Max steps"
)

# Organize the widgets in a vertical box layout
# The VBox contains the variable dropdown, level dropdown, robust checkbox,
# max steps slider, and a label with instructions
widgets.VBox([
    plot_pred_variable,
    plot_pred_level,
    plot_pred_robust,
    plot_pred_max_steps,
    widgets.Label(value="Run the next cell to plot the predictions. Rerunning this cell clears your selection.")
])

In [ ]:
# @title Plot predictions

plot_size = 5  # Set the plot size
plot_max_steps = min(predictions.dims["time"], plot_pred_max_steps.value)  # Determine the maximum steps to plot

# Prepare the data for plotting
data = {
    "Targets": scale(
        select(eval_targets, plot_pred_variable.value, plot_pred_level.value, plot_max_steps), 
        robust=plot_pred_robust.value
    ),
    "Predictions": scale(
        select(predictions, plot_pred_variable.value, plot_pred_level.value, plot_max_steps), 
        robust=plot_pred_robust.value
    ),
    "Diff": scale(
        (select(eval_targets, plot_pred_variable.value, plot_pred_level.value, plot_max_steps) -
         select(predictions, plot_pred_variable.value, plot_pred_level.value, plot_max_steps)),
        robust=plot_pred_robust.value, 
        center=0
    ),
}

# Set the figure title based on the selected variable and level
fig_title = plot_pred_variable.value
if "level" in predictions[plot_pred_variable.value].coords:
    fig_title += f" at {plot_pred_level.value} hPa"

# Plot the data
plot_data(data, fig_title, plot_size, plot_pred_robust.value)

# Train the model

The following operations require a large amount of memory and, depending on the accelerator being used, will only fit the very small "random" model on low resolution data. It uses the number of training steps selected above.

The first time executing the cell takes more time, as it include the time to jit the function.

In [ ]:
# @title Loss computation (autoregressive loss over multiple steps)
loss, diagnostics = loss_fn_jitted(
    rng=jax.random.PRNGKey(0),
    inputs=train_inputs,
    targets=train_targets,
    forcings=train_forcings)
print("Loss:", float(loss))

In [ ]:
# @title Gradient computation (backprop through time)
loss, diagnostics, next_state, grads = grads_fn_jitted(
    inputs=train_inputs,
    targets=train_targets,
    forcings=train_forcings)
mean_grad = np.mean(jax.tree_util.tree_flatten(jax.tree_util.tree_map(lambda x: np.abs(x).mean(), grads))[0])
print(f"Loss: {loss:.4f}, Mean |grad|: {mean_grad:.6f}")

In [ ]:
# @title Autoregressive rollout (keep the loop in JAX)
print("Inputs:  ", train_inputs.dims.mapping)
print("Targets: ", train_targets.dims.mapping)
print("Forcings:", train_forcings.dims.mapping)

predictions = run_forward_jitted(
    rng=jax.random.PRNGKey(0),
    inputs=train_inputs,
    targets_template=train_targets * np.nan,
    forcings=train_forcings)
predictions